# Competitive Pokemon Analysis

## Goal

## Results

## Updates

In [1]:
import pandas as pd
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

# Get Pokemon Stats CSV File
Pkmn_raw = pd.read_csv('Pokemon.csv')

In [2]:
# CLEAN POKEMON NAMES
# Assuming it's fine to keep as _raw
#Pandas Multiple Replace
#https://stackoverflow.com/questions/36072626/pandas-replace-multiple-values-at-once

Name_Replacements = {
    'Name':{
        # Remove duplicate names found in Mega, Primal, and Hoopa
        # Regex: ?= -> Look ahead Assertion
        # [Regex -> English] Replace everything before "Target Word" with empty space 
        # Pkmn_raw['Name'] = Pkmn_raw['Name'].str.replace(".*(?=Mega)", "")
        r'.*(?=Mega|Primal|Hoopa)': '',
        # Add Space between  Capitals to account in Forme, Cloake, Mode, Size
        # Regex:
        # [Regex -> English] Find two consecutive groups of (Words) and (Capital Letter) or ("50%") and Add a space between the two groups
        r'(\w)([A-Z])|(?=50%)': r'\1 \2'
    }
}

Pkmn_raw.replace(Name_Replacements, regex=True, inplace=True)

In [3]:
# CLEAN POKEMON GEN
# Issue: Mega Evolutions and Primal Reversions are Gen 6, but are appended to the respective Pokemon in the Pokedex

Pkmn_raw['Gen_Normalized'] = Pkmn_raw['Generation']

# CASE: Meganium -> NOT A MEGA EVOLUTION 
Pkmn_raw.loc[Pkmn_raw['Name'].str.contains('Mega |Primal '), 'Gen_Normalized'] = 6

#Pkmn_raw['Generation Normalized'] = Pkmn_raw['Generation'].replace(Gen_Replacements,regex=True)

# Make Generation Normalized = 6 for Mega and Primal Transform


In [4]:
# Box Plot of Base Stats per Generation (I-VI)
trace_all = go.Box(
                y = Pkmn_raw['Total'],
                name = 'All Gen', 
                boxpoints = 'all', 
                text = Pkmn_raw['Name'],
                pointpos = -2,
                jitter = 0.6,
                boxmean = 'sd',
                )

data = [trace_all]
for i in Pkmn_raw['Gen_Normalized'].sort_values().unique():
    trace = go.Box(
        y = Pkmn_raw.loc[Pkmn_raw['Gen_Normalized'] == i, 'Total'],
        name = "Gen " + str(i),
        boxpoints = 'all',
        text = Pkmn_raw.loc[Pkmn_raw['Gen_Normalized'] == i,'Name'],
        jitter = 0.6,
        boxmean = 'sd',
        pointpos = -2,
    )    
    data.append(trace)

layout = go.Layout(
    title = 'Total Stats Per Generation (I-VI)',
    autosize = False,
    yaxis = dict(range = [100, 900]),
    height = 2000,
    width = 4000,
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)



In [5]:
# POLAR SCATTER [CURRENTLY NOT IN USE!]
# Starter comparison
# 0 - 200
#         HP
#  ATTACK   SPECIAL ATTACK
#  DEFENSE  SPECIAL DEFENSE
#        SPEED
# HP ATTACK DEFENSE SP ATTACK SP DEF SPEED
# Set up plot [Check]
# Reorient Categories [Check]
# Value depending on pokemon name and column [Check]


def PkmnPolarStats(Pkmn_Comparison):
    data = []
    for Mon in Pkmn_Comparison:
        Stats_Polar = go.Scatterpolar(
            name = Mon,
            r = [
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'HP'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Sp. Atk'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Sp. Def'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Speed'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Defense'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Attack'], 
                 Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'HP']
                ],
            theta = ['HP','Sp. Atk','Sp. Def','Speed','Defense','Attack','HP'],
            fill = 'toself',
            hoveron = 'points',
        )
        data.append(Stats_Polar)
    return data

testdata1 = PkmnPolarStats(['Bulbasaur','Ivysaur','Squirtle','Wartortle','Charmander','Charmeleon'])
testdata2 = PkmnPolarStats(['Squirtle','Wartortle'])
testdata3 = PkmnPolarStats(['Charmander','Charmeleon'])

layout = go.Layout(   
    polar = dict(
        radialaxis = dict(
            range = [0,200],
        ),
        angularaxis = dict(
            rotation = 90,
        ),
    ),
    showlegend = True,
    title = 'TestTitle',
)


fig = go.Figure(data=testdata1,layout=layout)
py.iplot(fig, filename='polar-category')

In [ ]:
Pkmn_raw

In [6]:
# Creating Starter Dataframes from Raw Dataframe
Pkmn_Starter_Grass = ['Bulbasaur', 'Chikorita', 'Treecko', 'Turtwig', 'Snivy', 'Chespin']
Grass_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Grass)].drop(columns=['Generation','Legendary','Gen_Normalized'])

Pkmn_Starter_Fire = ['Charmander', 'Cyndaquil', 'Torchic', 'Chimchar', 'Tepig', 'Fennekin']
Fire_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Fire)].drop(columns=['Generation','Legendary','Gen_Normalized'])

Pkmn_Starter_Water = ['Squirtle', 'Totodile', 'Mudkip', 'Piplup', 'Oshawott', 'Froakie']
Water_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Water)].drop(columns=['Generation','Legendary','Gen_Normalized'])


In [ ]:
# Starter Comparison TEST
def ComparisonBoxPlot(Mon_df, Stats_list):
    data = []
    for Stat in Stats_list:
        Box_Trace= go.Box(
            y=Mon_df[Stat],
            name = Stat,
            boxpoints='all',
            text=Mon_df['Name'],
        ) 
    
        data.append(Box_Trace)
    return data

Pkmn_Stats = ['HP','Attack','Defense','Sp. Atk', 'Sp. Def', 'Speed']
Test_Grass = ComparisonBoxPlot(Grass_Starters, Pkmn_Stats)

layout = go.Layout(
    title='Test',
    autosize=True,
)
fig = go.Figure(data=Test_Grass, layout=layout)
py.iplot(fig)




In [7]:
# Starter Comparison Final
Stats_List=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
x = np.repeat(Stats_List,Pkmn_raw['Generation'].max()) #6 for All 6 generations of (grass/fire/water) type pokemen

trace0 = go.Box(
    y=pd.concat([Grass_Starters['HP'],
                 Grass_Starters['Attack'],
                 Grass_Starters['Defense'],
                 Grass_Starters['Sp. Atk'],
                 Grass_Starters['Sp. Def'],
                 Grass_Starters['Speed']
                ]),
    x=x,
    name='Grass',
    marker=dict(
        color='#78C850'
    ),
    boxpoints='all',
    text=pd.concat([Grass_Starters['Name']] * Pkmn_raw['Generation'].max())
)
trace1 = go.Box(
    y=pd.concat([Fire_Starters['HP'],
                 Fire_Starters['Attack'],
                 Fire_Starters['Defense'],
                 Fire_Starters['Sp. Atk'],
                 Fire_Starters['Sp. Def'],
                 Fire_Starters['Speed']
                ]),
    x=x,
    name='Fire',
    marker=dict(
        color='#F08030'
    ),
    boxpoints='all',
    text = pd.concat([Fire_Starters['Name']] * Pkmn_raw['Generation'].max())
)
trace2 = go.Box(
    y=pd.concat([Water_Starters['HP'],
                 Water_Starters['Attack'],
                 Water_Starters['Defense'],
                 Water_Starters['Sp. Atk'],
                 Water_Starters['Sp. Def'],
                 Water_Starters['Speed']
                ]),
    x=x,
    name='Water',
    marker=dict(
        color='#6890F0'
    ),
    boxpoints='all',
    text=pd.concat([Water_Starters['Name']] * Pkmn_raw['Generation'].max())
)
layout = go.Layout(
    title = 'Pokemon Starters',
    yaxis=dict(
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
py.iplot(fig)




In [ ]:
def PkmnGroupComparison(df,statslist)
    data = []
    y_box = []
    for stat in statslist: #HP Attack Defense #Sp. Atk #Sp. Def #Speed
        y_box.append(df[stat])
    
    trace = go.Box(
        y=y_box
        x=x_box
        boxpoints = 'all'
    )
    
    data.append(trace)
    return data



# $$ HP = \Bigg \lfloor \frac{ \big( 2 \times Base + IV + \big[ \frac{EV}{4} \big] \big) \times Level }{100} \Bigg\rfloor + Level + 10$$ 
# $$ Other Stat = \Bigg \lfloor \Bigg( \Bigg \lfloor\frac{ \big( 2 \times Base + IV + \big[ \frac{EV}{4} \big] \big) \times Level }{100} \Bigg \rfloor + 5 \Bigg) \times Nature \Bigg \rfloor$$
# $$ Damage = \Bigg( \frac { \big( \frac{ 2 \times Level}{5} +2 \big) \times Power \times A/D}{50} +2 \Bigg) \times Modifier$$
# $$ Modifier = Targets \times Weather \times Critical \times random \times STAB \times Type \times Burn \times Other$$

# $$ IV = [0, 31]$$
# $$ EV = [0, 252]$$
# $$ Level = [1, 100]$$
# A = Attack Value from Attacker (either Attack or Special Attack)
# D = Defense Value from Defender (either Defense or Special Defense)
# Nature is a 0.9 or 1.1 multiplier for specific stats (e.g. +Att - Def)
Flooring brackets (aka flooring function) indicates a "rounding down" of that value
e.g. from wikipedia floor(2.4) = 2 
e.g. floor(2.9999) = 2 

In [4]:
Pkmn_starters = pd.DataFrame([])
for i in Pkmn_raw['Gen_Normalized'].sort_values().unique():
    if i != 5:
        Pkmn_starters = Pkmn_starters.append(Pkmn_raw.loc[(Pkmn_raw['Gen_Normalized'] == i) & (Pkmn_raw['Generation']==Pkmn_raw['Gen_Normalized'])][0:9])
    # Gen 5 Victini CASE: Victini is the 1st Pokemon listed in Gen 5
    if i == 5:
        Pkmn_starters = Pkmn_starters.append(Pkmn_raw.loc[(Pkmn_raw['Gen_Normalized'] == i) & (Pkmn_raw['Generation']==Pkmn_raw['Gen_Normalized'])][1:10])
    
    
Pkmn_starters.drop(columns=['Legendary','Gen_Normalized'])
evo = pd.Series([1,2,3])
Pkmn_starters['Evolution_Stage'] = np.tile(evo, len(Pkmn_starters) //len(evo))


In [5]:
Pkmn_starters

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Gen_Normalized,Evolution_Stage
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1,1
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,1,2
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,1,3
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1,1
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,1,2
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,1,3
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,1,1
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False,1,2
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False,1,3
166,152,Chikorita,Grass,NaN,318,45,49,65,49,65,45,2,False,2,1


In [6]:
Stats_List=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
x = np.repeat(Stats_List,Pkmn_starters['Generation'].max()) #6 for All 6 generations of (grass/fire/water) type pokemen

# How to select by Type 1 and Evolution Stage
# .isin
#Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'HP']


# trace0 = go.Box(
#     y=pd.concat([Grass_Starters['HP'],
#                  Grass_Starters['Attack'],
#                  Grass_Starters['Defense'],
#                  Grass_Starters['Sp. Atk'],
#                  Grass_Starters['Sp. Def'],
#                  Grass_Starters['Speed']
#                 ]),
#     x=x,
#     name='Grass',
#     marker=dict(
#         color='#78C850'
#     ),
#     boxpoints='all',
#     text=pd.concat([Grass_Starters['Name']] * Pkmn_raw['Generation'].max())
# )
# trace1 = go.Box(
#     y=pd.concat([Fire_Starters['HP'],
#                  Fire_Starters['Attack'],
#                  Fire_Starters['Defense'],
#                  Fire_Starters['Sp. Atk'],
#                  Fire_Starters['Sp. Def'],
#                  Fire_Starters['Speed']
#                 ]),
#     x=x,
#     name='Fire',
#     marker=dict(
#         color='#F08030'
#     ),
#     boxpoints='all',
#     text = pd.concat([Fire_Starters['Name']] * Pkmn_raw['Generation'].max())
# )
# trace2 = go.Box(
#     y=pd.concat([Water_Starters['HP'],
#                  Water_Starters['Attack'],
#                  Water_Starters['Defense'],
#                  Water_Starters['Sp. Atk'],
#                  Water_Starters['Sp. Def'],
#                  Water_Starters['Speed']
#                 ]),
#     x=x,
#     name='Water',
#     marker=dict(
#         color='#6890F0'
#     ),
#     boxpoints='all',
#     text=pd.concat([Water_Starters['Name']] * Pkmn_raw['Generation'].max())
# )
# layout = go.Layout(
#     title = 'Pokemon Starters',
#     yaxis=dict(
#         zeroline=False
#     ),
#     boxmode='group'
# )

# fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
# py.iplot(fig)



array(['HP', 'HP', 'HP', 'HP', 'HP', 'HP', 'Attack', 'Attack', 'Attack',
       'Attack', 'Attack', 'Attack', 'Defense', 'Defense', 'Defense',
       'Defense', 'Defense', 'Defense', 'Sp. Atk', 'Sp. Atk', 'Sp. Atk',
       'Sp. Atk', 'Sp. Atk', 'Sp. Atk', 'Sp. Def', 'Sp. Def', 'Sp. Def',
       'Sp. Def', 'Sp. Def', 'Sp. Def', 'Speed', 'Speed', 'Speed',
       'Speed', 'Speed', 'Speed'], dtype='<U7')

In [11]:
Type = 'Grass' #[Grass, Fire, Water]
Evolution = 1 #[1,2,3]
Stat = 'Attack' #['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

Pkmn_starters.loc[(Pkmn_starters['Type 1'] == Type) & (Pkmn_starters['Evolution_Stage'] == Evolution),Stat]

0      49
166    49
272    45
432    68
554    45
718    61
Name: Attack, dtype: int64

In [8]:
import plotly.figure_factory as ff
# Add histogram data
x1 = Pkmn_raw['HP']
x2 = Pkmn_raw['Attack']
x3 = Pkmn_raw['Defense']
x4 = Pkmn_raw['Sp. Atk']
x5 = Pkmn_raw['Sp. Def']
x6 = Pkmn_raw['Speed']

# Group data together
hist_data = [x1, x2, x3, x4, x5, x6]

group_labels = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp.Def','Speed']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, show_curve=False)

# Plot!
py.iplot(fig, filename='Distplot with Multiple Datasets')